In [ ]:
# %pip install vertexai
import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project="talk-to-a-local-427009", location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-flash-001")

response = model.generate_content(
    "What's a good name for a flower shop that specializes in selling bouquets of dried flowers?"
)

print(response.text)

In [2]:
from openai import AsyncOpenAI
import json

keys = json.load(open("secrets.json"))

client = AsyncOpenAI(
    base_url="http://101.35.52.226:9090/v1",
    api_key=keys['api_key'],
    timeout=45,
)


async def chat(prompt, stream=False, temperature=0.0, n=1):
    response = await client.chat.completions.create(
        model="qwen-110b-chat",
        messages=[{"role": "user", "content": prompt}],
        stream=stream,
        max_tokens=512,
        temperature=temperature,
        n=n,
        stop=["<|endoftext|>", "<|im_end|>"],
    )
    if not stream:
        if n == 1:
            return response.choices[0].message.content.strip()
        return response.choices
    return response

In [11]:
# using the chat function, we need to prompt the LLM to extract relevant fields from the text
# this will create the major & minor descriptions table
import json
import pandas as pd

major_raw = json.load(open("major_details.json"))
major_links = pd.read_json('links_majors.json')
minor_raw = pd.read_json("minor_details.json")
minor_links = json.load(open('links_minors.json'))


In [108]:
# put all the texts into a list based on url
major = {}
for item in major_raw:
    if item['url'] not in major:
        major[item['url']] = []
    major[item['url']].append(item['text'].strip())

url = "https://www.engineering.cornell.edu/students/undergraduate-students/curriculum/majors-and-minors"
# url = "https://as.cornell.edu/major_minor_gradfield/classics"
# len(major[url]), major_links[major_links['link'] == url]['major'].values[0]
idx = 30
# [len(s) for s in set(major[url])]
# [len(s) for s in set(major[url])][idx:idx+5], list(set(major[url]))[idx:idx+5]
set(major)

# check if a certain portion of text is contained in one of those
# text = "If you have an interest in solving engineering problems related to human health"
# text = "As a chemical engineer"
# text = major[url][0]


# [(s if cond else "") for cond, s in [(text in s, s) for s in major[url]]]

{'https://aap.cornell.edu/academics/architecture',
 'https://aap.cornell.edu/academics/art',
 'https://as.cornell.edu/education/college-scholar-program',
 'https://as.cornell.edu/major_minor_gradfield/africana-studies',
 'https://as.cornell.edu/major_minor_gradfield/american-studies',
 'https://as.cornell.edu/major_minor_gradfield/anthropology',
 'https://as.cornell.edu/major_minor_gradfield/archaeology',
 'https://as.cornell.edu/major_minor_gradfield/asian-studies',
 'https://as.cornell.edu/major_minor_gradfield/astronomy',
 'https://as.cornell.edu/major_minor_gradfield/biology-society',
 'https://as.cornell.edu/major_minor_gradfield/chemistry-chemical-biology',
 'https://as.cornell.edu/major_minor_gradfield/china-asia-pacific-studies',
 'https://as.cornell.edu/major_minor_gradfield/classics',
 'https://as.cornell.edu/major_minor_gradfield/cognitive-science',
 'https://as.cornell.edu/major_minor_gradfield/comparative-literature',
 'https://as.cornell.edu/major_minor_gradfield/economic

In [106]:
def map_strings_to_big_string(text, strings):
    matches = []

    for s in strings:
        start_index = text.find(s)
        if start_index != -1:
            end_index = start_index + len(s) - 1
            matches.append((start_index, end_index))
        else:
            matches.append(None)

    total_characters = len(text)
    matched_characters = sum(len(s) for s in strings if s in text)

    return matched_characters / total_characters, matches


# Example usage
strings = [
    # "Majors and Minors | Cornell Engineering",
    # "Undergraduate SpotlightsStudents",
    # "DiversityPrograms & EventsCurrent Students",
    "Independent Major",
    "Materials Science and Engineering",
    "If you have an interest in solving engineering problems related to human health, consider the burgeoning field of",
    "biomedical engineering (BME). An integrated, interdisciplinary endeavor, BME bridges the engineering and medical fields to meet the 21st-century needs of biomedical",
]

matched_percentage, matches = map_strings_to_big_string(major[url][2], strings)
print(f"Matched rate: {matched_percentage}")
print(f"Matches: {matches}")
major[url][4]

Matched rate: 0.029188610193698117
Matches: [(6410, 6426), (7781, 7813), (1891, 2003), (2005, 2168)]


'HomeAdmissions▼Undergraduate AdmissionsGraduate AdmissionsUndergraduate SpotlightsStudents▼Undergraduate StudentsGraduate StudentsM.Eng. StudentsResearch & Faculty▼FacultyCenters and FacilitiesStrategic Areas of ResearchUndergraduate ResearchDiversity▼Programs & EventsCurrent StudentsProspective Students & OutreachCareer DevelopmentPartnershipsAbout the Office of Inclusive ExcellenceAlumni▼Bill Nye Solar Noon ClockFeatured AlumniAlumni EventsContact AADCornell Engineering StoreDistinguished Alumni AwardStudent SupportVolunteerOpen the search panelMenu'

In [ ]:
# what is the flow to extract this list of data? 
# the top most one def has every text from that website
# the ones below might have have junk contents like the nav bars, headings and stuff
# this is why we need to design a nice LLM prompt to extract the relevant data
# we can first prompt it to see if that piece has any value relevant to the application we are building: yes/no question if we should keep the data
# if yes, we can then prompt it to extract the relevant data

RELEVANCE_PROMPT = """
Is the following text relevant to the major/minor description?
{0}
"""

EXTRACT_PROMPT = """
Extract the relevant information from the following text:
{0}
"""